목표
--------
* url을 각각의 변수로 나누어서 실습한다.
* erd를 보고 필요한 내용들을 파싱해본다.
* 파싱한 데이터를 db에 저장한다.

In [44]:
#필요한 라이브러리 import
import requests
import urllib.request
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from pymysql import NULL
import pymysql
import re

In [3]:
conn = pymysql.connect(host="localhost", user='queant', password='queant1234', db = 'queant', charset='utf8mb4')
cur = conn.cursor()

In [3]:
#api활용할 url부분

#각 데이터 별 url 앞부분
url_deposit = ["https://finlife.fss.or.kr/finlifeapi/depositProductsSearch.xml?auth=", "https://finlife.fss.or.kr/finlifeapi/savingProductsSearch.xml?auth="]

#발급받은 키
api_key = "47c0e868fdb16333d47d0e385641c3c0"

#권역코드
code_front = "&topFinGrpNo="
code_bank = ["020000", "030300"]

#페이지
page_front = "&pageNo="
page_num = ["1", "2", "3", "4"]

In [229]:
url = url_deposit[1] + api_key + code_front + code_bank[1] + page_front + page_num[0]
print(url)
data_str = urllib.request.urlopen(url).read().decode('euc-kr')
data_xml = ET.fromstring(data_str)

https://finlife.fss.or.kr/finlifeapi/savingProductsSearch.xml?auth=47c0e868fdb16333d47d0e385641c3c0&topFinGrpNo=030300&pageNo=1


* 아래 코드는 예,적금 상품의 가입방법을 파싱하는 코드
    * join_way 태그에 접근해서 데이터를 받는다.
    * 기존의 데이터는 스마트폰, 인터넷, 영업점 등으로 표현되어있다.
    * DB에는 공통코드를 통해 저장할 것이기 때문에 각각의 가입방법 존재유무를 파악해야한다.
    * Api를 볼때 가입방법은 총 영업점, 인터넷, 스마트폰, 전화, 기타로 총 5가지이다.
    * 각각의 문자열이 join_way 데이터에 있는지 확인하고 그 유무를 True, False로 배열에 저장할 것.

In [ ]:
for product_tag in data_xml[5]:
    join = product_tag[0].find("join_way").text
    print(join)
    join_parsing = ["영업점" in join, "인터넷" in join, "스마트폰" in join, "전화" in join, "기타" in join]
    print(join_parsing)
    print()

* 아래 코드는 예,적금 상품의 가입 나이 제한을 파싱하는 코드
    * 데이터들을 보았을때 일정한 형식이 없다.
    * 이를 어떻게 파싱할지 계획하는 것이 중요.

예외적인 문구들
- 만19세~만34세 개인 및 개인사업자
- 만 10세 이하 자녀를 둔 부, 모
- 1954년 이전출생자 및 비과세종합저축가입대상자(만64세이상/국가유공자/국민기초생활보장법에의한수급자/장애인/독립유공자과그유족또는가족/고엽제후유증환자 및 5.18민주화운동부상자)
- 만 20~30대만 가입가능 (나이제한)
- 1~19세


<구상 1>
1. 과정
    1. 단어들 사이의 공백을 모두 없애서 다 붙인다.
    2. 만, 세 단어가 들어가는 문장들만 추출
    3. "이상"이라는 단어가 들어가는 문장에서 "만oo세이상" 부분 문자열의 oo부분만 추출한다.
    4. 마찬가지로 "이하", "초과" 단어가 들어가는 문장에서 나이만 뽑아온다.
2. 문제점
    1. 만 13세~ 만 24세라는 조건에서는 추출 불가능
    2. ~이상 ~이하 문구에서 추출하면 나이의 최소는 잘 추출되지만 최대 부분은 '만 ~세이상 만~' 식으로 추출됨.

<구상 2>
1. 과정
    1. 단어들 사이의 공백을 모두 없애서 다 붙인다.
    2. 문자열에서 숫자들만 추출한다.
2. 문제점
    1. 문자열에 있는 숫자가 무조건 나이라는 보장이 없다.

결론
- 거의 대부분의 조건들이 나이를 표기할 때 만~세 형식으로 사용
- 모든 만~세 문자열의 사이 부분을 추출해서 배열에 저장
- 배열길이가 1이면 이상, 이하, 미만, 초과 등의 단어에 따라 max,min값 저장
- 배열길이가 1이 아니면 정렬 후 앞의 데이터를 min에 뒤에 데이터를 max에 저장

- 해결해야할 부분: 만 ~세 미만 만~세 이상 일 경우에는 미만에서 '만'이라는 단어를 판단해서 오류 발생

In [182]:
def max_min_join(join_member):
    min_age = None
    max_age = None
    join_member = join_member.replace(" ", "")
    if "만" in join_member and "세" in join_member:
        ages = re.findall('만(.+?)세', join_member)
        ages = list(map(int, ages))
        if len(ages) == 1:
            if "이상" in join_member:
                min_age = ages[0]
            elif "초과" in join_member:
                min_age = ages[0] + 1
            elif "이하" in join_member:
                max_age = ages[0]
            elif "미만" in join_member:
                max_age = ages[0] - 1
        else:
            ages.sort()
            min_age = int(ages[0])
            max_age = int(ages[1])
    return min_age, max_age

        

In [230]:
for product_tag in data_xml[5]:
    join_deny = product_tag[0].find("join_deny").text
    join_member = product_tag[0].find("join_member").text
    min, max = max_min_join(join_member)
    print(join_member)
    print(min, max)
    print()

제한없음
NULL NULL

만19세이상 직장인
19 NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

만 19세 이상 실명의 개인고객 (1인 1계좌 한정)
19 NULL

누구나 가입가능
NULL NULL

누구나 가입가능
NULL NULL

누구나 가입가능
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

개인
NULL NULL

개인
NULL NULL

개인
NULL NULL

제한 없음
NULL NULL

개인
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

만17세이상
17 NULL

제한 없음
NULL NULL

제한 없음
NULL NULL

제한 없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

개인 (당행 첫거래 고객)
NULL NULL

개인 (직장인)
NULL NULL

개인
NULL NULL

개인 (만 7세 이하 미성년자)
NULL 7

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

만 20~30대만 가입가능 (나이제한)
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

만 19세 이상의 개인
19 NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

제한없음
NULL NULL

-제한없음
NULL NULL

-만19세이상

* 아래 코드는 가입 최소금액, 최대금액, 최대기간, 최소기간을 파싱할 코드
    * 가입 최소, 최대기간은 option 내용들로 활용할 예정

In [227]:
for product_tag in data_xml[5]:
    etc = product_tag[0].find("etc_note").text
    print(etc)
    print()

없음

없음

- 12~36개월 가입가능
- 가입금액 : 1만원이상~100만원이하
- KB착한e-plus정기적금과 납입한도 합산 적용

- 12~36개월 가입가능
- 가입금액 : 1만원이상

- 12~36개월 가입가능 - 가입금액 : 1만원이상~100만원이하 - KB착한정기적금과 납입한도 합산 적용

- 12~36개월 가입가능
- 가입금액 : 1만원이상

- 12개월 가입가능
- 가입금액 : 1만원이상~30만원이하
- 증빙서류제출후가입

- 12개월 가입가능 - 가입금액 : 1만원이상~20만원이하

- 최저가입금액 : 1만원이상 
- 중도해지금리 
- 1개월 미만 : 보통예금 금리 
- 1개월 이상 : 약정금리 x (경과월수 / 계약월수)x차등율 (최저금리 : 약정금리 x 50%) 
- 1개월 이상~12개월 미만(차등율 : 60%)
- 12개월 이상~24개월 미만(차등율 : 70%) 
- 24개월 이상(차등율 : 80%)

- 최저가입금액 : 1만원이상 
- 중도해지금리 
- 1개월 미만 : 보통예금 금리 
- 1개월 이상 : 약정금리 x (경과월수 / 계약월수)x차등율 (최저금리 : 약정금리 x 50%) 
- 1개월 이상~12개월 미만(차등율 : 60%) 
- 12개월 이상~24개월 미만(차등율 : 70%) 
- 24개월 이상(차등율 : 80%)

- 가입금액 : 1만원이상~100만원이하 
(중도해지금리)
- 1개월 미만 : 보통예금 금리 
- 1개월 이상 : 약정금리 x (경과월수 / 계약월수)x차등율 (최저금리 : 약정금리x 50%) 
- 1개월 이상~12개월 미만(차등율 : 60%) 
- 12개월 이상~24개월 미만(차등율 : 70%) 
- 24개월 이상(차등율 : 80%)

없음

- 1인당 가입한도 : 월 1만원~50만원 (1만원 단위로 가입 가능) 
- 가입기간 : 12개월 또는 24개월만 가능 
- 지점방문을 통한 상품가입만 가능 
- 반려견과 함께 찍은사진 직원 제시 필수 - 본인 외 대리인 개설 불가

없음

- 이자지급방식 : 만기일시지급


In [233]:
for product_tag in data_xml[5]:
    special = product_tag[0].find("spcl_cnd").text
    print(special)
    print()

-인터넷뱅킹 가입: 연 0.05% 
-폰뱅킹 가입: 연 0.05% 
-모바일뱅킹 가입 : 연 0.10%

최대 연 0.5% 우대금리 적용
※우대금리 적용조건
①첫거래고객(가입시점 당행에 적립식 및 거치식 예금 미보유 고객): 연 0.2%
②전자금융거래로 이체/신규/만기해지 7회이상(신규가입일~만기일): 연 0.1%
③개인정보 수집이용 동의(마케팅목적): 연 0.1% 
④인터넷뱅킹으로 신규 가입 시: 연 0.05%
⑤모바일뱅킹으로 신규 가입 시: 연 0.1%

없음

없음

없음

* 우대이율: 만기 유지시 만기축하 이율 연 0.40% 적용 - 기본이율 3.60% + 우대이율 0.40% ※ 우대이율은 중도해지시에는 적용되지 않습니다. - 자세한 내용은 당사 모바일뱅킹 APP을 통해 확인하여 주시기 바랍니다.

* 최고우대금리 : 0.2% 
- 인터넷 가입시 : 연 0.1% 
- 모바일 가입시 : 연 0.2%

* 최고우대금리 : 0.2% 
- 인터넷 가입시 : 연 0.1% 
- 모바일 가입시 : 연 0.2%

* 최고우대금리 : 0.2% 
- 인터넷 가입시 : 연 0.1% 
- 모바일 가입시 : 연 0.2%

* 인터넷(스마트뱅킹포함) 6개월~36개월 창구금리+0.1%

없음

없음

- 창구가입 - 3명 이상 창구 방문하여 1년이상 적금 가입시 : 연 0.1%
- 적금 만기해지 후 1년이상 정기예금 및 적금 신규 가입시 : 연 0.1%

없음

없음

없음

-

-

없음

없음

없음

-

-

-

없음

없음

- 최고우대금리 : 0.1%
- 자영업자, 직장인(증빙서류제출-재직증명서, 사업자등록증, 명함, 사원카드) : 0.1%

- 최고우대금리 : 0.1%
- 자영업자, 직장인(증빙서류제출-재직증명서, 사업자등록증, 명함, 사원카드) : 0.1%

없음

- 인터넷뱅킹(스마트뱅킹 포함)으로 가입 시 : 연0.1% 
- SBI패밀리 멤버쉽 최초 가입 당일 적금 가입 시 1인 1계좌 우대금리 적용 : 연 0.1%

- 만기우대금리 : 0.5% 
- 자동

* 아래 코드는 옵션 부분들을 어떻게 파싱할지 구상하는 코드
    * 필요한 변수
        * base_rate: 기본 금리
        * save_trm: 저축 기간
        * rate_type: 금리 유형명 (단리, 복리)
        * rsrv_type: 저축방식 (자유적립, 정액적립)
    * 공통코드 활용 안하기 때문에 직접 저장하면 된다.

In [226]:
for product_tag in data_xml[5]:
    print("new prdt")
    for option_tag in product_tag[1]:
        rate = option_tag.find("intr_rate").text
        save_term = option_tag.find("save_trm").text
        rate_type = option_tag.find("intr_rate_type_nm").text
        rsrv_type = option_tag.find("rsrv_type_nm").text
        print(rate, save_term, rate_type, rsrv_type)

new prdt
2.3 12 단리 정액적립식
2.5 24 단리 정액적립식
2.6 36 단리 정액적립식
2 6 단리 정액적립식
new prdt
2.3 12 단리 정액적립식
2.4 24 단리 정액적립식
2.5 36 단리 정액적립식
2 6 단리 정액적립식
new prdt
2.4 12 단리 정액적립식
2.5 24 단리 정액적립식
2.6 36 단리 정액적립식
new prdt
2.2 36 단리 정액적립식
2.1 24 단리 정액적립식
2 12 단리 정액적립식
new prdt
2.5 12 단리 정액적립식
2.6 24 단리 정액적립식
3 36 단리 정액적립식
new prdt
2.1 12 단리 정액적립식
2.2 24 단리 정액적립식
2.3 36 단리 정액적립식
new prdt
4.5 12 단리 정액적립식
new prdt
1.5 12 단리 정액적립식
new prdt
2 6 단리 정액적립식
2.5 36 단리 정액적립식
2.4 24 단리 정액적립식
2.3 12 단리 정액적립식
new prdt
2.4 12 단리 정액적립식
2.5 24 단리 정액적립식
2.1 6 단리 정액적립식
new prdt
2.5 24 단리 정액적립식
2.6 36 단리 정액적립식
new prdt
2.6 12 단리 정액적립식
2.7 24 단리 정액적립식
2.7 36 단리 정액적립식
2 6 단리 정액적립식
new prdt
3.1 24 단리 정액적립식
3 12 단리 정액적립식
new prdt
2.7 12 단리 정액적립식
2.8 24 단리 정액적립식
2.8 36 단리 정액적립식
2 6 단리 정액적립식
new prdt
2.2 12 단리 정액적립식
2.3 24 단리 정액적립식
2.4 36 단리 정액적립식
new prdt
2.3 12 단리 정액적립식
2.4 24 단리 정액적립식
2.5 36 단리 정액적립식
new prdt
1.5 36 단리 자유적립식
1.5 24 단리 자유적립식
1.5 12 단리 자유적립식
new prdt
1.6 12 단리 자유적립식
1.6 24 단리 자유적립식
1.6 36 단리 자유적립식
new prdt
2.3